In [1]:
import os
from samgeo import tms_to_geotiff, split_raster
from samgeo.text_sam import LangSAM

In [2]:
# imagem

image = '/Users/jac/Documents/temp/imgs_drone_praia_brava/100_mosaico_recorte.tif'
# image = '/Users/jac/Documents/temp/imgs_drone_praia_brava/101_georref/DJI_0001_georeferenced.png'

In [3]:
# criar diretórios se não existirem

def check_and_create_directories(pasta):
    print(pasta)
    # Testar pela existência de pasta_saida e criar se não existir
    if not os.path.exists(pasta):
        os.makedirs(pasta)

In [4]:
# argumentos

text_prompt = "person"

box_t = 0.4

text_t = 0.4

pasta_saida = f'saida/processamento_mosaico/{text_prompt.replace(" ", "_")}_{box_t}_{text_t}'

pasta_tiles = f'{pasta_saida}/tiles'

mascaras_saida = f'{pasta_saida}/masks'

check_and_create_directories(pasta_saida)
check_and_create_directories(pasta_tiles)
check_and_create_directories(mascaras_saida)

saida/processamento_mosaico/person_0.4_0.4
saida/processamento_mosaico/person_0.4_0.4/tiles
saida/processamento_mosaico/person_0.4_0.4/masks


In [5]:
# dividir em pedaços menores, lidar com restrições de memória

split_raster(image, out_dir=pasta_tiles, tile_size=(1000, 1000), overlap=0)

In [6]:
# carregar o modelo de segmentação
# escolha do modelo
# vit_b(91M), vit_l(308M), vit_h(636M)

sam = LangSAM(model_type='vit_b')

final text_encoder_type: bert-base-uncased


In [7]:
mascaras = sam.predict_batch(
    images=pasta_tiles,
    out_dir=mascaras_saida,
    text_prompt=text_prompt,
    box_threshold=box_t,
    text_threshold=text_t,
    mask_multiplier=255,
    dtype='uint8',
    merge=True,
    verbose=True
)

Processing image 01 of 35: saida/processamento_mosaico/person_0.4_0.4/tiles/tile_0_0.tif...
Processing image 02 of 35: saida/processamento_mosaico/person_0.4_0.4/tiles/tile_0_1.tif...
No objects found in the image.
Processing image 03 of 35: saida/processamento_mosaico/person_0.4_0.4/tiles/tile_0_2.tif...
No objects found in the image.
Processing image 04 of 35: saida/processamento_mosaico/person_0.4_0.4/tiles/tile_0_3.tif...
No objects found in the image.
Processing image 05 of 35: saida/processamento_mosaico/person_0.4_0.4/tiles/tile_0_4.tif...
No objects found in the image.
Processing image 06 of 35: saida/processamento_mosaico/person_0.4_0.4/tiles/tile_1_0.tif...
No objects found in the image.
Processing image 07 of 35: saida/processamento_mosaico/person_0.4_0.4/tiles/tile_1_1.tif...
Processing image 08 of 35: saida/processamento_mosaico/person_0.4_0.4/tiles/tile_1_2.tif...
Processing image 09 of 35: saida/processamento_mosaico/person_0.4_0.4/tiles/tile_1_3.tif...
No objects found 

In [8]:
# exportar a máscara para um shapefile

sam.raster_to_vector(f'{mascaras_saida}/merged.tif',
                     f'{pasta_saida}/{text_prompt.replace(" ", "_")}_{box_t}_{text_t}.shp')